In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares
import winsound
import nidaqmx
import pandas as pd

# If the imports don't work, press F1, find "Developer: Reload Window"

factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title4

#rm = pyvisa.ResourceManager()
#rm.list_resources()


In [ ]:
# DMM = rm.open_resource('GPIB0::23::INSTR')
# DMM.write("Reset; End")
# DMM.read_termination = "\r\n"
# DMM.timeout = 5000

# DMM.clear()
# NPLC = 10
# DMM.write(f"NDIG 8")
# DMM.write(f"NPLC {NPLC}")
# DMM.write("TARM AUTO")
# DMM.write("DCI")
# DMM.write("APER 1E-1")
#DMM.write("DISP OFF")


11

In [4]:
device_name = "dev1"
DAQ_read_channels = ["ai0","ai1","ai2", "ai3"]
DAQ_write_channel = "ao0"
DAQ_timeout = 5000

### Data collection functions

In [5]:
# RAMP TO VOLTAGE 


def ramp_to(start_v, end_v, total_ramp_time=600, v_step=0.001):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.
    
    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return
    
    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(voltage_range / v_step)  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step
    
    # Set the initial voltage to start_v
    current_voltage = start_v
    
    try:
        with nidaqmx.Task() as task:
            # Configure the AO0 channel on device "dev1"
            task.ao_channels.add_ao_voltage_chan("dev1/ao0", min_val=0.0, max_val=10.1)
            
            # Set the initial voltage to start_v
            task.write(current_voltage)
            print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")
            
            # Determine the direction of the ramp (up or down)
            step_direction = 1 if end_v > start_v else -1
            
            for _ in range(num_steps):
                # Update the current voltage in the direction of the ramp
                current_voltage += step_direction * v_step
                
                # Output the new voltage to the DAQ
                task.write(current_voltage)
                
                # Wait for the calculated time step
                time.sleep(dt)
                
                # Check for error conditions 
                error_bits = DMM.query("ERR?")
                if error_bits != 0:  # Placeholder for an actual error condition
                    print("Error: A condition occurred. Exiting ramp. Eror_bits:")
                    print(error_bits)
                    break
                
                # Placeholder for additional error checking or exception handling
                # You can add custom error handling code here
                pass

    except Exception as e:
        # If any exception occurs, handle it here
        print(f"An exception occurred: {str(e)}. Exiting ramp.")
        return

    print(f"Successfully ramped from {start_v}V to {current_voltage}V.")


# Example usage:
# ramp_to(0.5, 5.0)  # Ramps from 0.5V to 5.0V over 60 seconds with a step size of 0.001V




# RAMP TO BUT SAVE DAQ AND MULTIMETER DATA

def ramp_and_record(start_v, end_v, total_ramp_time=600, v_step=0.001, filename="ramp_and_record"):
    """
    Ramps the voltage on the DAQ output from start_v to end_v over total_ramp_time, using steps of v_step.

    Parameters:
        start_v (float): Starting voltage, must be between 0 and 10.1.
        end_v (float): Ending voltage, must be between 0 and 10.1.
        total_ramp_time (float): Total time for the voltage ramp (default is 60 seconds).
        v_step (float): Voltage step (default is 0.001 volts).
    """
    # Check if input voltages are within the valid range
    if not (0 <= start_v <= 10.1) or not (0 <= end_v <= 10.1):
        print("Error: Voltage must be between 0 and 10.1 volts.")
        return

    # Calculate the total number of steps and the time interval (dt) between steps
    voltage_range = abs(end_v - start_v)
    num_steps = int(np.ceil(voltage_range / v_step))  # Number of voltage steps
    dt = total_ramp_time / num_steps  # Time between each voltage step

    # DMM settings
    #DMM_data = []
    #t0=time.time()

    #t_const = 8E-2
    #if t_const <= dt*0.9:
        #DMM.write(f"APER {t_const}")
    #else:
        #DMM.write(f"APER {dt*0.9}")

    # Set the initial voltage to start_v
    current_voltage = start_v

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)*((num_steps + 1)/(num_steps))))

    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Data logging for ramp started
        #time.sleep(0.1)

        # Start the DAQ task for data logging
        read_task.start()


        try:
            with nidaqmx.Task() as task:
                # Configure the AO0 channel on device "dev1"
                task.ao_channels.add_ao_voltage_chan("dev1/ao0", min_val=0.0, max_val=10.1)

                # Set the initial voltage to start_v
                task.write(current_voltage)
                print(f"Starting ramp from {start_v}V to {end_v}V with step {v_step}V.")

                # Determine the direction of the ramp (up or down)
                step_direction = 1 if end_v > start_v else -1

                for _ in range(num_steps):

                    time.sleep(dt)
                    #temp_timer = time.time()
                    #while (time.time() - temp_timer) < dt:
                        #DMM_data.append([time.time()-t0, float(DMM.read())])

                    # Wait for the calculated time step
                    #time.sleep(dt)

                    # Update the current voltage in the direction of the ramp
                    current_voltage += step_direction * v_step

                    # Output the new voltage to the DAQ
                    task.write(current_voltage)
                    print(f"current voltage: {current_voltage}V.")

                    # Check for error conditions
                    #error_bits = DMM.query("ERR?")
                    #if int(error_bits) != 0:  # Placeholder for an actual error condition
                    #    print("Error: A condition occurred. Exiting ramp. Error_bits:")
                    #    print(error_bits)
                    #    return

                    # Placeholder for additional error checking or exception handling
                    pass

        except Exception as e:
            # If any exception occurs, handle it here
            print(f"An exception occurred: {str(e)}. Exiting ramp.")
            return
        
        time.sleep(dt)

        #temp_timer = time.time()
        #while (time.time() - temp_timer) < dt:
        #    DMM_data.append([time.time()-t0, float(DMM.read())])


        print(f"Successfully ramped from {start_v}V to {current_voltage}V.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon_neg': DAQ_data[0],
        'I_mon_neg': DAQ_data[1],
        'V_mon_pos': DAQ_data[2],
        'I_mon_pos': DAQ_data[3],
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_{start_v:.3f}V_to_{end_v:.3f}V_step_{v_step}V_in_{total_ramp_time}s_{timestamp}_DAQ.csv"
    df.to_csv(full_file_name, index=False)

    # Convert to a pandas DataFrame
    #df2 = pd.DataFrame({
    #    'DMM_Elapsed_Time': [point[0] for point in DMM_data],
    #    'DMM_Leakage_Current': [point[1] for point in DMM_data]
    #})

    # Save to CSV for multimeter data
    #full_file_name2 = f"{filename}_{start_v:.3f}V_to_{end_v:.3f}V_step_{v_step}V_in_{total_ramp_time}s_{timestamp}_Keysight.csv"
    #df2.to_csv(full_file_name2, index=False)

    

# Example usage:
# ramp_and_record(0.1, 0, 30, 0.001)  # Ramps from 0.1V to 0V over 30 seconds with a step size of 0.001V


def record(total_ramp_time=600, filename="record"):

    # DMM settings
    #DMM_data = []
    t0=time.time()

    # DAQ settings
    sample_rate = 10  # Samples per second
    num_samples = int((sample_rate * (total_ramp_time * 1.1)))
    timestamp = time.strftime('%Y-%m-%d_%H-%M-%S')

    with nidaqmx.Task() as read_task:
        for ch in DAQ_read_channels:
            read_task.ai_channels.add_ai_voltage_chan(f"{device_name}/{ch}")

        read_task.timing.cfg_samp_clk_timing(sample_rate, samps_per_chan=num_samples)

        # Start the DAQ task for data logging
        read_task.start()

        time.sleep(total_ramp_time)

        #temp_timer = time.time()
        #while (time.time() - temp_timer) < total_ramp_time:
        #    DMM_data.append([time.time()-t0, float(DMM.read())])

        print(f"Successfully recorded.")

        # Read the data with the specified timeout
        DAQ_data = read_task.read(number_of_samples_per_channel=num_samples, timeout=DAQ_timeout)
        elapsed_time = [i / sample_rate for i in range(num_samples)]

    # Convert to a pandas DataFrame
    df = pd.DataFrame({
        'DAQ_Elapsed_Time': elapsed_time,
        'V_mon_neg': DAQ_data[0],
        'I_mon_neg': DAQ_data[1],
        'V_mon_pos': DAQ_data[2],
        'I_mon_pos': DAQ_data[3],
    })

    # Save to CSV for multimeter data
    full_file_name = f"{filename}_recording_{total_ramp_time}s_{timestamp}_DAQ.csv"
    df.to_csv(full_file_name, index=False)

    # Convert to a pandas DataFrame
    #df2 = pd.DataFrame({
    #    'DMM_Elapsed_Time': [point[0] for point in DMM_data],
    #    'DMM_Leakage_Current': [point[1] for point in DMM_data]
    #})

    # Save to CSV for multimeter data
    #full_file_name2 = f"{filename}_recording_{total_ramp_time}s_{timestamp}_Keysight.csv"
    #df2.to_csv(full_file_name2, index=False)

    


In [15]:
ramp_and_record(9.,10.,100,0.01)

Starting ramp from 9.0V to 10.0V with step 0.01V.
current voltage: 9.01V.
current voltage: 9.02V.
current voltage: 9.03V.
current voltage: 9.04V.
current voltage: 9.049999999999999V.
current voltage: 9.059999999999999V.
current voltage: 9.069999999999999V.
current voltage: 9.079999999999998V.
current voltage: 9.089999999999998V.
current voltage: 9.099999999999998V.
current voltage: 9.109999999999998V.
current voltage: 9.119999999999997V.
current voltage: 9.129999999999997V.
current voltage: 9.139999999999997V.
current voltage: 9.149999999999997V.
current voltage: 9.159999999999997V.
current voltage: 9.169999999999996V.
current voltage: 9.179999999999996V.
current voltage: 9.189999999999996V.
current voltage: 9.199999999999996V.
current voltage: 9.209999999999996V.
current voltage: 9.219999999999995V.
current voltage: 9.229999999999995V.
current voltage: 9.239999999999995V.
current voltage: 9.249999999999995V.
current voltage: 9.259999999999994V.
current voltage: 9.269999999999994V.
cur

In [ ]:
record(3600,"rec_at_10")